# Grid Explorer

This example allows you to interactively explore the spectra in a `synthesizer` grid.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from synthesizer.grid import Grid

try:
    import ipywidgets as widgets
except ImportError:
    import sys

    print(
        "ipywidgets not found. Please install with pip install ipywidgets"
        " and run jupyter nbextension enable --py widgetsnbextension "
        "to use this notebook."
    )
    sys.exit(0)

In [ ]:
grid_dir = "../../tests/test_grid/"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

print(grid)

In [ ]:
def update_spectra(
    log10age_grid_point=0,
    metallicity_grid_point=0,
    spectra_id="incident",
    wavelength_range=[2.5, 4.0],
):
    """
    Function to make a plot of the spectra.

    Arguments:
        log10age_grid_point (int)
            The age grid point.
        metallicity_grid_point (int)
            The metallicity grid point.
        spectra_id (str)
            The id of the spectra to be plotted.
        wavelength_range (list, float)
            The log10(wavelength/Angstrom) range to plot
    """

    # define grid point - set by widget
    grid_point = (log10age_grid_point, metallicity_grid_point)

    # get sed object - spectra_id set by widget
    sed = grid.get_spectra(grid_point, spectra_id=spectra_id)

    # plot sed
    plt.plot(
        np.log10(sed.lam),
        np.log10(sed.lnu),
        lw=1,
        alpha=0.8,
    )

    # add labels for age and metallicity
    plt.figtext(
        0.15,
        0.2,
        rf"$\log_{{10}}(age/Myr): {grid.log10age[log10age_grid_point]:.2f}$",
    )
    plt.figtext(
        0.15, 0.15, rf"$Z: {grid.metallicity[metallicity_grid_point]}$"
    )

    # define log10 wavelength range - set by widget
    plt.xlim(wavelength_range)

    # define log10 luminosity range
    plt.ylim([15.0, 22.0])

    # add axes labels
    plt.xlabel(r"$\log_{10}(\lambda/\AA)$")
    plt.ylabel(r"$\log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1}\ M_{\odot}^{-1})$")

    # show
    plt.show()

    # alternative implementation using plot_spectra
    # plot_spectra(
    #     sed,
    #     ylimits=10**np.array([15.0, 22.0]),
    #     xlimits=10**np.array(wavelength_range),
    #     )


# spectra selection widget
spectra_id = widgets.Dropdown(
    options=["incident", "nebular", "total"],
    value="incident",
    description="spectra type",
    disabled=False,
)

# wavelength range selection widget
wavelength_range = widgets.FloatRangeSlider(
    value=[2.5, 4.0],
    min=0,
    max=6.0,
    step=0.1,
    description="Wavelength range:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".1f",
)

# age index selection widget
age = widgets.IntSlider(
    min=0, max=len(grid.log10age) - 1, step=1, value=0, description="age index"
)

# metallicity index selection widget
metallicity = widgets.IntSlider(
    min=0,
    max=len(grid.metallicity) - 1,
    step=1,
    value=0,
    description="metallicity index",
)

# define UI
ui = widgets.VBox([wavelength_range, spectra_id, age, metallicity])

# collect widgets
out = widgets.interactive_output(
    update_spectra,
    {
        "log10age_grid_point": age,
        "metallicity_grid_point": metallicity,
        "spectra_id": spectra_id,
        "wavelength_range": wavelength_range,
    },
)

# display
display(ui, out)